In [1]:
# Import libraries
from bs4 import BeautifulSoup
import requests
import time
import datetime
import smtplib

In [2]:
# Connect to and pull data from website

URL = 'https://www.amazon.com/Pokemon-Brilliant-Diamond-Nintendo-Switch/dp/B08Y6HCHP8/ref=sr_1_8?crid=301JQLH8OC8L&keywords=pokemon&qid=1643226962&sprefix=pokemon%2Caps%2C126&sr=8-8'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36", "Accept-Encoding":"gzip, deflate, br", "Accept-Language": "en-US,en;q=0.9", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

page = requests.get(URL, headers=headers)

soup = BeautifulSoup(page.text, "lxml")

soup_pretty = BeautifulSoup(soup.prettify(), "lxml")
#print(soup_pretty)

title = soup_pretty.find(id="productTitle").get_text()

# After inspecting the website, the id ("priceblock_ourprice") did not match the id listed in soup_pretty (base-product-price)
# Thus, need to find price based on soup_pretty
price = soup_pretty.find(id="base-product-price")["data-base-product-price"]


print(title)
print(price)


           Pokémon Brilliant Diamond - Nintendo Switch
          
$54.99


In [3]:
# Clean data

title = title.strip()
price = float(price.strip()[1:])

print(title)
print(price)

Pokémon Brilliant Diamond - Nintendo Switch
54.99


In [4]:
# Create Timestamp to track when data was collected

output_date = datetime.date.today()

print(output_date)

2022-01-26


In [5]:
# Write headers and data into CSV file

import csv

header = ["Date", "Title", "Price"]
data = [output_date, title, price]

# To import initial data into csv
with open("Amazon_Pokemon_Scraper_Dataset.csv", "w", newline='', encoding="UTF8") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

In [6]:
# Read and check CSV

import pandas as pd

df = pd.read_csv('Amazon_Pokemon_Scraper_Dataset.csv')

print(df)

         Date                                        Title  Price
0  2022-01-26  Pokémon Brilliant Diamond - Nintendo Switch  54.99


In [7]:
# Define function to send email to self when price drops

def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    server.ehlo()
    server.login('kimberlyhnguyen98@gmail.com','xxxxxxxxxxxxxx')
    
    subject = "Pokemon Brilliant Diamond is now under $50! Now is your chance to buy!"
    body = "Go, Kimberly! Now is your chance to pick up a new Pokemon game. Link here: https://www.amazon.com/Pokemon-Brilliant-Diamond-Nintendo-Switch/dp/B08Y6HCHP8/ref=sr_1_8?crid=301JQLH8OC8L&keywords=pokemon&qid=1643226962&sprefix=pokemon%2Caps%2C126&sr=8-8"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'kimberlyhnguyen98@gmail.com',
        msg
     
    )

In [10]:
# Combine all code from above into one function

def check_pokemon_price():
    URL = 'https://www.amazon.com/Pokemon-Brilliant-Diamond-Nintendo-Switch/dp/B08Y6HCHP8/ref=sr_1_8?crid=301JQLH8OC8L&keywords=pokemon&qid=1643226962&sprefix=pokemon%2Caps%2C126&sr=8-8'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36", "Accept-Encoding":"gzip, deflate, br", "Accept-Language": "en-US,en;q=0.9", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers=headers)

    soup = BeautifulSoup(page.text, "lxml")

    soup_pretty = BeautifulSoup(soup.prettify(), "lxml")
    
    title = soup_pretty.find(id="productTitle").get_text()
    
    price = soup_pretty.find(id="base-product-price")["data-base-product-price"]
    
    output_date = datetime.date.today()
    
    # Clean data
    
    title = title.strip()
    price = float(price.strip()[1:])
    
    import csv

    header = ["Date", "Title", "Price"]
    data = [output_date, title, price]
    
    # Append data to CSV
    with open("Amazon_Pokemon_Scraper_Dataset.csv", "a+", newline='', encoding="UTF8") as f:
        writer = csv.writer(f)
        writer.writerow(data)

    # Sending email to self if price drops
    if(price < 50.00):
        send_mail()

In [ ]:
# Run function every 24 hours

while(True):
    check_pokemon_price()
    time.sleep(86400)